### Data Cleaning

In [20]:
import re

# Read in the data
with open('/content/drive/MyDrive/mini project/tokenized_data.txt', 'r') as f:
    data = f.readlines()

# Define the cleaning function
def clean_sentence(sentence):
    # Remove all non-alphanumeric characters
    cleaned_sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence)
    # Convert the sentence to lowercase
    cleaned_sentence = cleaned_sentence.lower()
    # Remove leading and trailing whitespace
    cleaned_sentence = cleaned_sentence.strip()
    # Replace any remaining whitespace with a single space character
    cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence)
    return cleaned_sentence

# Clean the data
cleaned_data = []
for sentence in data:
    cleaned_sentence = clean_sentence(sentence)
    cleaned_data.append(cleaned_sentence)

# Print the first 5 cleaned sentences
for sentence in cleaned_data[:5]:
    print(sentence)


resumption of the session reprise de la session
i declare resumed the session of the european parliament adjourned on friday 17 december 1999 and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period je dclare reprise la session du parlement europen qui avait t interrompue le vendredi 17 dcembre dernier et je vous renouvelle tous mes vux en esprant que vous avez pass de bonnes vacances
although as you will have seen the dreaded millennium bug failed to materialise still the people in a number of countries suffered a series of natural disasters that truly were dreadful comme vous avez pu le constater le grand bogue de lan 2000 ne sest pas produit en revanche les citoyens dun certain nombre de nos pays ont t victimes de catastrophes naturelles qui ont vraiment t terribles
you have requested a debate on this subject in the course of the next few days during this partsession vous avez souhait un dbat ce sujet dans les prochains jours au

### Semi-supervised training data selection

In [11]:
pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 KB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━

In [13]:
import torch
import torchtext.datasets as datasets
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
import torchdata.datasets as datasets


# Define the tokenizer
tokenizer = get_tokenizer('basic_english')

# Load the monolingual data
train_data, test_data = AG_NEWS(root='data', split=('train', 'test'))


# Tokenize the data
train_tokens = []
for example in train_data:
    tokens = tokenizer(example.text)
    train_tokens.append(tokens)

# Train a language model on the data
vocab = torchtext.vocab.build_vocab_from_iterator(train_tokens)
train_dataset = LanguageModelingDataset(train_tokens, vocab, unk_token=vocab.unk_index)
model = torch.nn.LSTM(input_size=vocab.vectors.shape[1], hidden_size=256, num_layers=2, dropout=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
for epoch in range(10):
    for batch in torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True):
        inputs = batch[:-1]
        targets = batch[1:]
        outputs, _ = model(inputs)
        loss = loss_fn(outputs.view(-1, vocab.vectors.shape[1]), targets.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Select the most useful sentences for machine translation
translation_model = torch.nn.TransformerEncoderLayer(d_model=vocab.vectors.shape[1], nhead=8, dim_feedforward=512)
translation_model.load_state_dict(model.state_dict())
scores = []
for tokens in train_tokens:
    inputs = torch.tensor(vocab.lookup_indices(tokens))
    outputs, _ = translation_model(inputs.unsqueeze(1))
    scores.append(outputs[-1].mean().item())
top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:1000]
selected_data = [train_data[i].text for i in top_indices]

# Save the selected data to a file
with open('selected_data.txt', 'w') as f:
    f.writelines(selected_data)


Exception: ignored

## Data Augmentation

In [18]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.2 MB/s eta 0:00:00


In [15]:
pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 21.7 MB/s eta 0:00:00


In [19]:
import nlpaug.augmenter.word as naw
import nlpaug.flow as naf
import random

# Define the data augmentation pipeline
aug_pipeline = naf.Sequential([
    naw.SynonymAug(aug_src='wordnet'),
    naw.RandomWordAug(action='substitute'),
    naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")
])

# Load the training data
with open('/content/drive/MyDrive/mini project/tokenized_data.pickle', 'r') as f:
    data = f.readlines()

# Augment the data
aug_data = []
for example in data:
    aug_example = aug_pipeline.augment(example)
    aug_data.append(aug_example)

# Combine the original data with the augmented data
combined_data = data + aug_data

# Shuffle the combined data
random.shuffle(combined_data)

# Save the augmented data to a file
with open('augmented_data.txt', 'w') as f:
    f.writelines(combined_data)


UnicodeDecodeError: ignored